In [1]:
import sys

sys.path.append('/home/milo/repos/obia')


In [2]:
from obia.handlers import open_geotiff
from obia.segment import segment

In [ ]:
raster_path = "merged122.tif"

image = open_geotiff(raster_path)

In [3]:
from osgeo import gdal, osr
import rasterio
import numpy as np

def open_binary_raster_as_mask(mask_path):
    raster = gdal.Open(mask_path, gdal.GA_ReadOnly)
    band = raster.GetRasterBand(1)
    mask_array = band.ReadAsArray().astype(bool)
    return mask_array

In [ ]:
mask_path = "chm_122_gt2.tif"
mask = open_binary_raster_as_mask(mask_path)

In [ ]:
segmented_image = segment(
    image, segmentation_bands=[7,8,4], statistics_bands=[7],
    calc_mean=True, calc_variance=True, calc_skewness=False, calc_kurtosis=False, 
    calc_contrast=True, calc_dissimilarity=False, calc_homogeneity=False, calc_ASM=False, calc_energy=False, calc_correlation=True,
    method="slic", n_segments=6000, compactness=0.01, max_num_iter=100, sigma=0, convert2lab=False, slic_zero=True, mask=mask
)

In [ ]:
segmented_image.write_segments('segments_6000_h.gpkg')

In [4]:
import os
from rasterio.mask import mask
import geopandas as gpd

In [5]:
tile_index_path = 'tileindex.gpkg'
raster1_path = 'merged.tif'
raster2_path = 'chm_aligned_full_gt3.tif'
output_dir = 'clipped'

def clip_and_save(raster_path, output_path, geometry):
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, geometry, crop=True)
        out_meta = src.meta.copy()
        out_meta.update(
            {
                'driver': 'GTiff',
                'height': out_image.shape[1],
                'width': out_image.shape[2],
                'transform': out_transform
            }
        )
        with rasterio.open(output_path, 'w', **out_meta) as dest:
            dest.write(out_image)
                
os.makedirs(output_dir, exist_ok=True)

tile_index = gpd.read_file(tile_index_path)

for idx, tile in tile_index.iterrows():
    tile_geometry = [tile['geometry']]

    output_raster1 = os.path.join(output_dir, f'img_tile_{idx}.tif')
    output_raster2 = os.path.join(output_dir, f'chm_tile_{idx}.tif')

    clip_and_save(raster1_path, output_raster1, tile_geometry)
    clip_and_save(raster2_path, output_raster2, tile_geometry)


In [6]:
import math

7563

In [ ]:
for i in range(tile_index.shape[0]):
    raster_path = f"clipped/img_tile_{i}.tif"
    mask_path = f"clipped/chm_tile_{i}.tif"
    
    image = open_geotiff(raster_path)
    mask = open_binary_raster_as_mask(mask_path)

    pixel_area = 0.5 ** 2
    crown_area = math.pi * (5 ** 2) 
    tree_area = mask.sum() * pixel_area
    n_crowns = round(tree_area/crown_area)

    segmented_image = segment(
        image, segmentation_bands=[7,8,4], statistics_bands=[7],
        calc_mean=True, calc_variance=True, calc_skewness=False, calc_kurtosis=False, 
        calc_contrast=True, calc_dissimilarity=False, calc_homogeneity=False, calc_ASM=False, calc_energy=False, calc_correlation=True,
        method="slic", n_segments=n_crowns, compactness=0.01, max_num_iter=100, sigma=0, convert2lab=False, slic_zero=True, mask=mask
    )
    segmented_image.write_segments(f'segments/seg_tile_{i}.gpkg')

/home/milo/miniconda3/envs/obia/lib/python3.11/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
100%|██████████| 133/133 [00:22<00:00,  5.81it/s]
/home/milo/miniconda3/envs/obia/lib/python3.11/site-packages/skimage/segmentation/slic_superpixels.py:60: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  centroids, _ = kmeans2(coord[idx_dense], coord[idx], iter=5)
100%|██████████| 425/425 [01:14<00:00,  5.68it/s]
/home/milo/miniconda3/envs/obia/lib/python3.11/site-packages/skimage/segmentation/slic_superpixels.py:60: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  centroids, _ = kmeans2(coord[idx_dense], coord[idx], iter=5)
100%|██████████| 349/349 [00:30<00:00, 11.44it/s]
/home/milo/miniconda3/envs/obia/lib/python3.11/site-packages/skimage/segmentation/